In [21]:
from langchain_google_genai import GoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
import os
from typing import TypedDict
from langchain_core.prompts import PromptTemplate

In [22]:
load_dotenv()

True

In [23]:
model = GoogleGenerativeAI(
    model = 'gemini-2.5-flash',
    google_api_key = os.getenv('GEMINI_API_KEY')
)

In [24]:
class LLMState(TypedDict):
    question : str
    answer : str
    

In [31]:
def llm_qa(state: LLMState) -> LLMState:

    # Extract the question from the state
    question = state['question']

    # Form a prompt
    prompt = f"Give the answer to the question: {question}"

    # Ask that question to the llm
    answer = model.invoke(prompt)

    # Get the output and partially update the state
    state['answer'] = answer

    return state

In [32]:
#Now we will be creating our graph
graph = StateGraph(LLMState)

graph.add_node('llm_qa', llm_qa)

graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

workflow = graph.compile()

In [33]:
#Execute 

initial_state = {'question': 'What is the name of the first planet of the Solar System?'}

final_state = workflow.invoke(initial_state)
print(final_state)

{'question': 'What is the name of the first planet of the Solar System?', 'answer': 'The first planet of the Solar System, closest to the Sun, is **Mercury**.'}
